In [1]:
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# Loading excel file to assign labels to images

data_df = pd.read_csv('driver_imgs_list.csv')

# Image folder path
images_folder = 'limited_dataset'

In [3]:
# Mapping the classname codes to to full names

class_mapping = {
    'c0' : 'Safe driving',
    'c1' : 'Texting - right',
    'c2' : 'Talking on the phone - right',
    'c3' : 'Texting - left',
    'c4' : 'Talking on the phone - left',
    'c5' : 'Operating the radio',
    'c6' : 'Drinking', 
    'c7' : 'Reaching behind',
    'c8' : 'Hair and makeup',
    'c9' : 'Talking to passenger'
}

data_df['classname'] = data_df['classname'].map(class_mapping)

In [4]:
# Preprocess the images and split the dataset

np.random.seed(333)

datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

img_height = 180
img_width = 180
batch_size = 8

train_ds = datagen.flow_from_dataframe(
    dataframe = data_df,
    directory = images_folder,
    x_col = 'img',
    y_col = 'classname',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    subset = 'training'
)

valid_ds = datagen.flow_from_dataframe(
    dataframe = data_df,
    directory = images_folder,
    x_col = 'img',
    y_col = 'classname',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    subset = 'validation'
)

Found 160 validated image filenames belonging to 10 classes.


c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 22224 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  warnings.warn(


Found 40 validated image filenames belonging to 10 classes.


c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\preprocessing\image.py:1137: UserWarning: Found 22224 invalid image filename(s) in x_col="img". These filename(s) will be ignored.
  warnings.warn(


In [8]:
print(train_ds.labels)

[4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7, 4, 9, 6, 8, 5, 2, 0, 3, 1, 7]


In [10]:
len(train_ds.labels)

160

## Model 1
First model attempt with minimal data preprocessing

In [5]:
 # First model 
model_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation = 'relu')
])

In [6]:
model_1.compile(optimizer = 'adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy']
              )

In [7]:
epochs = 10
history = model_1.fit(
    train_ds,
    validation_data = valid_ds,
    epochs = epochs
)

Epoch 1/10




InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main

  File "c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code

  File "C:\Users\brian\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\brian\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 737, in start

  File "c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\site-packages\tornado\platform\asyncio.py", line 215, in start

  File "c:\Users\brian\AppData\Local\Programs\Python\Python39\lib\asyncio\base_eve

In [ ]:
# Visualize training results

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()